In [ ]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy.signal import cwt
from scipy.io import loadmat

# Load iEEG data
lfp = dict()
mat_file_path = '/Users/notz/Desktop/Projects/mouse/Complt_sig.mat'
data = loadmat(mat_file_path)
lfp['pre'] = data['pre']
lfp['are'] = data['are']
lfp['gab'] = data['gab']

# General analysis parameters
trl_num = lfp['pre'].shape[0]
cnhl_num = lfp['pre'].shape[1]
Fs = 500
Time = np.arange(1, lfp['pre'].shape[2] + 1) / Fs

# Demo
trl = 901
plt.figure()
for chnl in range(4):
    x = lfp['pre'][trl, chnl, :]
    clr = [0, 1 - 1/(chnl + 1), 1 - 1/(chnl + 1)]
    plt.plot(Time, chnl + x, color=clr)

plt.show()

# Drug comparison
clr1 = [0.5, 0.8, 0.8]
clr2 = [0.5, 0.5, 0.5]
clr3 = [0.8, 0.5, 0.5]

drug_strn = ['Pregabalin', 'Gabapentin', 'Arecoline']
trl = 90
chnl = 2

plt.figure(figsize=(10, 6))

# Pregabalin
plt.subplot(3, 3, 1)
x = lfp['pre'][trl, chnl, :]
plt.plot(Time, x, color=clr1, linewidth=2)
plt.title(drug_strn[0])
plt.box(False)

# Gabapentin
plt.subplot(3, 3, 4)
x = lfp['gab'][trl, chnl, :]
plt.plot(Time, x, color=clr2, linewidth=2)
plt.title(f'Similar Drug ({drug_strn[1]})')
plt.box(False)

# Arecoline
plt.subplot(3, 3, 7)
x = lfp['are'][trl, chnl, :]
plt.plot(Time, x, color=clr3, linewidth=2)
plt.title('Other Drug (Arecoline)')
plt.box(False)

plt.tight_layout()
plt.show()
width = np.array(2)
# Calculate Power Spectrogram
power = []

for drug in range(3):
   signal = lfp['pre'] if drug == 0 else (lfp['gab'] if drug == 1 else lfp['are'])
# Convert to an array

for trl in range(trl_num // 5):
    
    y = signal[trl, chnl, :]
    cfs, frq = pywt.cwt(y, np.arange(1,128), 'mexh' )
    power.append(cfs)

# Plot Power Spectrum
plt.figure(figsize=(10, 6))
eqlzr1 = []
eqlzr2 = []

for drug in range(3):
    plt.subplot(3, 3, 3 * drug + 1)
    avg_pwr = np.abs(np.nanmean(power[drug::3], axis=0).T)
    start_freq = np.argmin(np.abs(frq - 300))
    A = avg_pwr[:, start_freq:]
    plt.pcolormesh(Time, frq[start_freq:], A, shading='auto', cmap='RdBu_r', vmax=np.max(A), vmin=np.min(A))
#    plt.imshow(Time, extent=[-1, 64, 1, 2], cmap='PRGn', aspect='auto',
 #          vmax=np.max(A), vmin=np.min(A))  
    eqlzr1.append(np.max(A))
    eqlzr2.append(np.min(A))
    plt.yscale('log')
    plt.ylabel('Frequency')
    plt.xlabel('Trial Time (s)')
    plt.colorbar()

for drug in range(3):
    plt.subplot(3, 3, 3 * drug + 1)
    plt.clim(np.min(eqlzr2), np.max(eqlzr1))

plt.show()